# Joint detection for physical attacks and FDI attacks
1 Import any 3-phase unbalanced distribution network (with PV inverters and VV/VW controls) from OpenDSS (in .dss format)
2 Train/test GCN to detect 
  1) whether each PV inverter at each timepoint is under physical attack
  2) whether each measurement sensor at each timepoint is under FDI attack

In [ ]:
# 0 import
from data_loader import RolloutStorage, data_loader_FDIPhy
from graph_loader import single2batch_phy
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from cplxmodule import cplx
from cplxmodule.nn.modules import CplxConv1d
from cplxmodule.nn.modules import CplxLinear
from model.layers import ChebGraphConv
from model.utility import calc_gso, calc_chebynet_gso, cnv_sparse_mat_to_coo_tensor
from cplxmodule.nn import CplxToCplx
from torch.autograd import Variable
import opendssdirect as dss
import os
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sps
from scipy.sparse import csc_matrix, diags
import random
from sklearn.metrics import roc_curve, auc 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
random.seed(100) 
dss.Basic.ClearAll() # Initialize OpenDSS

# 1 check GPU
# Check if CUDA is available
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    # Get the current device
    current_device = torch.cuda.current_device()
    print("Current CUDA Device Index:", current_device)
    # Get device name
    print("Current CUDA Device Name:", torch.cuda.get_device_name(current_device))
    # Check CUDA version
    print("CUDA Version:", torch.version.cuda)
# check version of numpy and torch
print("NumPy version:", np.__version__)
print("PyTorch version:", torch.__version__)
# check if NumPy is available
try:
    arr = np.array([1, 2, 3], dtype=np.float32)
    print("NumPy array:", arr)
except Exception as e:
    print("NumPy error:", e)
# check if Numpy is compatible with PyTorch
try:
    tensor = torch.from_numpy(arr)
    print("Torch tensor:", tensor)
except Exception as e:
    print("Torch error:", e)

In [ ]:
# 2 initialize OpenDSS for a selected 3-phase unbalanced distribution system
current_dir = os.getcwd()
#print("current work folder:", current_dir)
area = 'P4U'
sce = 'scenarios'
timeseries = 'base_timeseries'
loadshape = 'opendss_no_loadshapes'
substation = 'p4uhs0_4'
feeder = 'p4uhs0_4--p4udt4'
Master_dss = os.path.join(area, sce, timeseries, loadshape, substation, feeder, 'Master.dss')

result = dss.run_command("Redirect "+Master_dss)
print('finished initial load') #Timeseries runs do not include solve in them

NumBus = dss.Circuit.NumBuses()
NumNode = dss.Circuit.NumNodes()
print(NumBus)
print(NumNode)

In [ ]:
# 3 extract Y matrix from OpenDSS
Y_NodeOrder = dss.Circuit.YNodeOrder()
metadata = np.load("metadata_run_config.npy", allow_pickle=True).item()
sensor_locations = metadata["sensor_nodes"]
Y_NodeOrder = metadata["Y_node_order"]
sensor_location_indices = [Y_NodeOrder.index(s) for s in sensor_locations]
Y_sparse = sps.csc_matrix(dss.YMatrix.getYsparse())
# Y_sparse type is scipy.sparse.csc_matrix
# e.g., Y_sparse = csc_matrix((data, (row, col)), shape=(n, n))
# Extract diagonal matrix from Ybus
Dmat = diags(Y_sparse.diagonal(), format='csc')
# Compute the inverse square root of the diagonal matrix
Dmat_inv_sqrt = diags(1 / np.sqrt(Dmat.diagonal()), format='csc')
# Normalize the admittance matrix
Y_norm = Dmat_inv_sqrt @ Y_sparse @ Dmat_inv_sqrt
# Filter small values in Y_norm
Y_norm = Y_norm.toarray()  # Convert to dense matrix for element-wise operations
real_small = np.abs(Y_norm.real) < 1e-8
imag_small = np.abs(Y_norm.imag) < 1e-8
Y_norm.real[real_small] = 0
Y_norm.imag[imag_small] = 0
# Convert back to sparse format
Y_norm_sparse = csc_matrix(Y_norm)

In [ ]:
file_path1 = os.path.join(current_dir, 'Vphasor_FDI_Physical_WithVoltageNoise.npy')
file_path2 = os.path.join(current_dir, 'AttackLabel_FDI_Physical_WithVoltageNoise.npy')

data_obtain = data_loader_FDIPhy(
    file_path1=file_path1,
    file_path2=file_path2
)
# === Load shape info from data ===
VoltagePhasor = np.load(file_path1)  # shape: (num_time, num_node, Twindow)
AttackLabel = np.load(file_path2)    # shape: (num_time, num_labels)

num_time, Dimbus, Twindow = VoltagePhasor.shape
num_time2, Dimlabel = AttackLabel.shape

assert num_time == num_time2, "Mismatch in number of timepoints"



In [ ]:
# 6 Neural Network Design
CplxReLU = CplxToCplx[torch.nn.ReLU]
class ActorCriticGCN_CNN(nn.Module):
    def __init__(self, input_shape, label_size):
        super(ActorCriticGCN_CNN, self).__init__()
        self.Channel_T = 10
        self.Channel_S = 10
        self.conv1 = CplxConv1d(in_channels=input_shape[1], out_channels=self.Channel_T, kernel_size=1)
        K_order = 6
        n_feat = input_shape[1]
        enable_bias = True
        self.conv2 = ChebGraphConv(K_order, self.Channel_T, self.Channel_S, enable_bias)  
        self.fc1 = CplxLinear(self.Channel_T * input_shape[0], 512)
        init_ = lambda m: self.layer_init(m, nn.init.orthogonal_,
          lambda x: nn.init.constant_(x, 0))
        self.linear_forecast = init_(nn.Linear(512*2, label_size))   # 32, 1  ---> 2, 1
        self.activate = nn.Sigmoid()
    def forward(self, x, GSO):
        batch_or_agent, nodes_num,  T_num= x.shape
        x = x.permute(0, 2, 1)  # [Batch or agent, time, bus]
        x = self.conv1(x)  # [batch, Channel_T, bus]
        x = CplxReLU()(x)  # modrelu(input, threshold=0.5)
        x = x.permute(0, 2, 1) # [batch, bus, Channel_T]
        x = x.reshape(-1, self.Channel_T)  # [batch * bus, Channel_T]
        # x_im = torch.zeros(x.size(0), x.size(1))
        x = self.conv2(x, GSO)# [batch * bus, Tnum]
        x = CplxReLU()(x)  # modrelu(input, threshold=0.5)
        x = x.view(-1, nodes_num, self.Channel_T) # [batch, bus, Channel_T]
        x = x.view(x.size(0), -1)  # [batch, bus*Channel_T]
        x = self.fc1(x) # [batch, 512]
        x = CplxReLU()(x)  # modrelu(input, threshold=0.5)
        x_cat = torch.cat((x.real, x.imag), 1)
        value = self.linear_forecast(x_cat)
        value = self.activate(value)
        return value
    def layer_init(self, module, weight_init, bias_init, gain=1):
        weight_init(module.weight.data, gain=gain)
        bias_init(module.bias.data)
        return module

In [ ]:
# 7 GCN initialization
Twindow = 20
obs_shape    = (Dimbus, Twindow) 
target_shape = (Dimlabel, 1) 
model = ActorCriticGCN_CNN(obs_shape, Dimlabel).to(device)
batch_size = 200
total_train_num = 25000
num_mini_batch = 10
ppo_epoch = 3
rollouts = RolloutStorage(batch_size, obs_shape, target_shape, device)
gso_type = "sym_norm_lap"
edge_index, edge_weight = single2batch_phy(Y_norm_sparse, num_mini_batch) 
GSO = calc_gso(edge_index, edge_weight, Dimbus * num_mini_batch, gso_type)
GSO = cnv_sparse_mat_to_coo_tensor(GSO, device)
num_epochs = 50
learning_rate = 1e-4 

In [ ]:
# 8 If needed, then train GCN
FlagTrain = 1
if FlagTrain == 1:
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    criterion =  nn.BCELoss()
    for epoch in range(num_epochs):
        time_counter_input = 1
        time_counter_output = 1
        loss_mean = []
        for batch_num in range(total_train_num//batch_size):
            for i in range(batch_size):
                label_truth, data_feature = data_obtain.next_state(time_counter_input, time_counter_output)
                time_counter_input += 1
                time_counter_output += 1
                # current_obs = update_current_obs(data_t_rec.reshape(-1, 1))
                data_feature = torch.from_numpy(data_feature.astype(np.csingle)).to(device)  
                current_label = label_truth.reshape(-1, 1)
                current_label = torch.from_numpy(current_label).to(device)
                rollouts.insert(data_feature, current_label)
            for e in range(ppo_epoch):
                data_generator = rollouts.batch_generator(num_mini_batch)
                for sample in data_generator:
                    observations_batch, target_batch = sample # observations_batch.shape = [10, 118, 10] target_batch = [10, 118, 1]
                    # observations_batch_rm = torch.cat((observations_batch.real, observations_batch.imag), 1).reshape(num_mini_batch, -1)
                    # ===================forward=====================
                    input = Variable(observations_batch).to(device)
                    output = model(input, GSO)
                    target_batch = target_batch.reshape(target_batch.shape[0], -1)
                    #loss = criterion(output, target_batch)
                    output_physical = output[:, :30]
                    output_fdi = output[:, 30:]
                    target_physical = target_batch[:, :30]
                    target_fdi = target_batch[:, 30:]
                    loss_physical = criterion(output_physical, target_physical)
                    loss_fdi = criterion(output_fdi, target_fdi)
                    lambda_physical = 4
                    lambda_fdi = 1.0
                    loss = lambda_physical * loss_physical + lambda_fdi * loss_fdi
                    
                    # ===================backward====================
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    loss_mean.append(loss.item())
        torch.save(model.state_dict(), 'FDIPhyAtkDet_156_New.pth')
        # val_loss = test_func(obs_shape)
        loss_array = np.mean(np.array(loss_mean))

In [ ]:
# 9 GCN testing
def cal_acc(a,b):
    n=a.shape[0]
    m=a.shape[1]
    tterr=0
    r_err=0
    for i in range(n):
        cuerr=0
        for j in range(m):
            if a[i][j]!= b[i][j]:
               tterr+=1
               cuerr+=1
        if cuerr>0:
            r_err+=1
    return 1-r_err/n, 1-tterr/(n*m)

def accuracy_test(pred_y, true_y):
    pred_bin = pred_y.copy()
    batch, dim = pred_bin.shape
    for i in range(batch):
        for j in range(dim):
            if pred_bin[i][j] > 0.5:
                pred_bin[i][j] = 1
            else:
                pred_bin[i][j] = 0
    row, acca = cal_acc(pred_bin, true_y)
    return row, acca

test_rollouts = RolloutStorage(batch_size, obs_shape, target_shape, device)
model_testing =  ActorCriticGCN_CNN(obs_shape, Dimlabel)
model_testing.load_state_dict(torch.load('FDIPhyAtkDet_156_New.pth', map_location=torch.device('cpu')))
model_testing.to(device)
model_testing.eval()
test_start_point = total_train_num + 1000
time_counter_input = test_start_point
time_counter_output = test_start_point
accuracy_mean = []
# test_start_point + total_num_testing < 35040  
total_num_testing = 6000   
batch_size = 200
num_mini_batch = 10
# ROC
all_predictions = []
all_true = []

with torch.no_grad():
    for batch_num in range(total_num_testing//batch_size):
        accuracy_mean = []
        for i in range(batch_size):
            label_truth, data_feature = data_obtain.next_state(time_counter_input, time_counter_output)
            time_counter_input += 1
            time_counter_output += 1
            # current_obs = update_current_obs(data_t_rec.reshape(-1, 1))
            data_feature = torch.from_numpy(data_feature.astype(np.csingle)).to(device) 
            current_label = label_truth.reshape(-1, 1)
            current_label = torch.from_numpy(current_label).to(device)
            test_rollouts.insert(data_feature, current_label)
        data_generator = test_rollouts.batch_generator(num_mini_batch)
        for sample in data_generator:
            observations_batch, target_batch = sample
            # ===================forward=====================
            input = Variable(observations_batch).to(device)
            output = model_testing(input, GSO)
            target_batch = target_batch.reshape(target_batch.shape[0], -1)
            all_predictions.append(output.cpu().detach().numpy())
            all_true.append(target_batch.cpu().detach().numpy())
            
            _, accuracy = accuracy_test(output.cpu().detach().numpy(), target_batch.cpu().detach().numpy())
            # ===================backward====================
            accuracy_mean.append(accuracy)
        loss_array = np.mean(np.array(accuracy_mean))                

In [ ]:
# 10 ROC curve plotting
y_pred = np.concatenate(all_predictions, axis=0)
y_true = np.concatenate(all_true, axis=0)
#print(np.shape(y_pred))
#print(np.shape(y_true))
y_pred_flat = y_pred.flatten()
y_true_flat = y_true.flatten()
#print(np.shape(y_pred_flat))
#print(np.shape(y_true_flat))

fpr, tpr, _ = roc_curve(y_true_flat, y_pred_flat)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.8f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for both Physical & FDI Attacks')
plt.legend(loc='lower right')
#plt.savefig('ROC_AUC.png', dpi=300) 
plt.show()

In [ ]:
# First 30 are physical attack
y_pred_physical = y_pred[:, :30]
y_true_physical = y_true[:, :30]
y_pred_physical_flat = y_pred_physical.flatten()
y_true_physical_flat = y_true_physical.flatten()
# physical attack ROC
fpr_physical, tpr_physical, _ = roc_curve(y_true_physical_flat, y_pred_physical_flat)
roc_auc_physical = auc(fpr_physical, tpr_physical)
plt.figure(figsize=(8, 6))
plt.plot(fpr_physical, tpr_physical, color='darkorange', lw=2,
         label='Physical Attack ROC (AUC = %0.8f)' % roc_auc_physical)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Physical Attack (on 30 Inverters)')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Last 120 are FDI attack
y_pred_fdi = y_pred[:, 30:]
y_true_fdi = y_true[:, 30:]
y_pred_fdi_flat = y_pred_fdi.flatten()
y_true_fdi_flat = y_true_fdi.flatten()
# FDI attack ROC 
fpr_fdi, tpr_fdi, _ = roc_curve(y_true_fdi_flat, y_pred_fdi_flat)
roc_auc_fdi = auc(fpr_fdi, tpr_fdi)
plt.figure(figsize=(8, 6))
plt.plot(fpr_fdi, tpr_fdi, color='darkorange', lw=2,
         label='FDI Attack ROC (AUC = %0.8f)' % roc_auc_fdi)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for FDI Attack (on 120 Sensors)')
plt.legend(loc='lower right')
plt.show()